In [1]:
from tqdm import tqdm
from glob import glob
import json
import random

random.random()

0.6200042656152798

In [2]:
with open('train.json', 'w') as fopen_jsonl:
    with open('translated-mnli-train.jsonl') as fopen:
        for l in tqdm(fopen):
            data = json.loads(l)
            label = data['gold_label']
            if label == '-':
                continue
            sent1 = data['sentence1'].strip()
            sent2 = data['sentence2'].strip()
            
            sent1_ms = data['translate'][0].strip()
            sent2_ms = data['translate'][1].strip()
            
            left = f'ayat1: {sent1} ayat2: {sent2}'
            d = {"src": left, "label": label}
            fopen_jsonl.write(f'{json.dumps(d)}\n')
            
            left = f'ayat1: {sent1} ayat2: {sent2_ms}'
            d = {"src": left, "label": label}
            fopen_jsonl.write(f'{json.dumps(d)}\n')
            
            left = f'ayat1: {sent1_ms} ayat2: {sent2_ms}'
            d = {"src": left, "label": label}
            fopen_jsonl.write(f'{json.dumps(d)}\n')
            
            left = f'ayat1: {sent1_ms} ayat2: {sent2}'
            d = {"src": left, "label": label}
            fopen_jsonl.write(f'{json.dumps(d)}\n')

392702it [00:04, 89806.14it/s]


In [3]:
!shuf train.json > shuffled-train.json

In [4]:
with open('test.json', 'w') as fopen_jsonl:
    with open('translated-mnli-dev_matched.jsonl') as fopen:
        for l in tqdm(fopen):
            data = json.loads(l)
            label = data['gold_label']
            if label == '-':
                continue
            sent1 = data['sentence1'].strip()
            sent2 = data['sentence2'].strip()
            
            sent1_ms = data['translate'][0].strip()
            sent2_ms = data['translate'][1].strip()
            
            left = f'ayat1: {sent1} ayat2: {sent2}'
            d = {"src": left, "label": label}
            fopen_jsonl.write(f'{json.dumps(d)}\n')
            
            left = f'ayat1: {sent1} ayat2: {sent2_ms}'
            d = {"src": left, "label": label}
            fopen_jsonl.write(f'{json.dumps(d)}\n')
            
            left = f'ayat1: {sent1_ms} ayat2: {sent2_ms}'
            d = {"src": left, "label": label}
            fopen_jsonl.write(f'{json.dumps(d)}\n')
            
            left = f'ayat1: {sent1_ms} ayat2: {sent2}'
            d = {"src": left, "label": label}
            fopen_jsonl.write(f'{json.dumps(d)}\n')

10000it [00:00, 83295.51it/s]


In [5]:
!shuf test.json > shuffled-test.json

In [6]:
!head -n 1000 shuffled-test.json > test-1k.json

In [ ]:
from datasets import load_dataset

In [ ]:
data_files = {"train": 'train.json', "validation": 'train.json'}
raw_datasets = load_dataset(
    "json",
    data_files=data_files,
)

In [ ]:
label_list = raw_datasets["train"].unique("label")
label_list.sort()  # Let's sort it for determinism
num_labels = len(label_list)
label_to_id = {v: i for i, v in enumerate(label_list)}
label_to_id

In [ ]:
from transformers import T5Model, T5Tokenizer, T5PreTrainedModel, T5Config, AutoConfig, AutoModelForSequenceClassification
from transformers.models.bart.modeling_bart import shift_tokens_right

In [ ]:
tokenizer = T5Tokenizer.from_pretrained("mesolitica/t5-super-tiny-bahasa-cased")
config = T5Config.from_pretrained("mesolitica/t5-super-tiny-bahasa-cased",
                                  num_labels=3)

In [ ]:
def preprocess_function(examples):
    inputs = [ex for ex in examples['src']]

    result = tokenizer(inputs, max_length=256, padding='longest', truncation=True)
    # Map labels to IDs (not necessary for GLUE tasks)
    if label_to_id is not None and "label" in examples:
        result["label"] = [(label_to_id[ex] if ex != -1 else -1) for ex in examples['label']]
    return result

In [ ]:
raw_datasets = raw_datasets.map(
    preprocess_function,
    batched=True,
)